# ISMRM2021

# Simulation Configs

- *n* = 10000 Signals
- *Std* = 0.1
- *Model*: s(b)/s(0) = (1-f)*exp(-bD)+ f*exp(-bD*)
- *Params*: **D** - Diffusion Coefficient, **D*** - Pseudodiffusion Coefficient and **f** - Perfusion Fraction 
- *Params Values*: **D** = 0.00081, **D*** = 0.022 and **f** = 0.1 Mean values of Gray Matter from doi:10.2463/mrms.mp.2019-0061

## Linear Algorithm
![image.png](images/img1.png)

## Combinatory Algorithm

![image-2.png](images/img2.png)

### Examples:

- For the reduction of 14 bvals to 9 bvals (b = 0 is a common b-value), we have:
- Linear Algorithm: (14+13+12+11+10) = 60 iterations (60 fittings with different b-values samplings)
- Combinatory Algorithm: $\frac{14!}{9!(14-9)!}$ = 2002 iterations

# Code

In [1]:
# Importing Linear and Combinatory Algorithm
from algorithms.algorithms import linear_reduction, combinatory_reduction

# Import Simulation Modules
import numpy as np

# Simulation
def simul_signal4d(params_mean, std, shape, bvals):
    """ 4D Matrix are equivalent to a real image .nii
    """

    d_mean, pd_mean, f_mean = params_mean
    data = np.zeros((shape[0],shape[1],shape[2],shape[3]))
    signal = (1-f_mean)*np.exp(-bvals*d_mean)+f_mean*np.exp(-bvals*pd_mean)
    
    for i in range(shape[0]):
            for j in range(shape[1]):
                    for k in range(shape[2]):
                        noise = np.random.normal(1, std, len(signal))
                        data[i,j,k,:] = (signal*noise)
    return data

# Simulations Configs
params_mean = [0.00081, 0.022, 0.1] # Mean values of Gray Matter from doi:10.2463/mrms.mp.2019-0061
bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500,
                  1000, 1200, 1400, 1600, 1800, 2000]) # The algorithm will ignore the bvalue = 0.
std = 0.1 
n = 5 # = n-r (number of reduced b-values)

shape = [10,1,1,len(bvals)] # 4D matrix 100x100x1x15 bvals - 10 000 "voxels".

# Create a simulate signal
sig = simul_signal4d(params_mean, std, shape, bvals)

#print(sig[0,0,0])

# RUN LINEAR ALGORITHM - Rsquared
best_bvals = linear_reduction(n, bvals, sig)

print(best_bvals)

100%|██████████| 600/600 [00:07<00:00, 76.42it/s]

[0, 4, 8, 16, 120, 1000, 1400, 1600, 1800, 2000]


In [2]:
# RUN COMBINATORY ALGORITHM - Rsquared
best_bvals = combinatory_reduction(n, bvals, sig)

print(best_bvals)

100%|██████████| 20020/20020 [03:43<00:00, 89.64it/s]

[0, 8, 30, 250, 1000, 1200, 1400, 1600, 1800, 2000]


In [3]:
# Replicate with std = 0.25
std = 0.25
sig = simul_signal4d(params_mean, std, shape, bvals)

# RUN LINEAR ALGORITHM - Rsquared
best_bvals = linear_reduction(n, bvals, sig)

print(best_bvals)

# RUN COMBINATORY ALGORITHM - Rsquared
best_bvals = combinatory_reduction(n, bvals, sig)

print(best_bvals)

  0%|          | 10/20020 [00:00<03:35, 93.00it/s]

[0, 8, 16, 60, 1000, 1200, 1400, 1600, 1800, 2000]


100%|██████████| 20020/20020 [03:40<00:00, 90.71it/s]

[0, 8, 16, 120, 1000, 1200, 1400, 1600, 1800, 2000]


In [ ]:
# Codeberg Repository: https://codeberg.org/lcscosta/Ithaca

In [ ]:
# Simulations Configs
params_mean = [0.00081, 0.022, 0.1] # Mean values of Gray Matter from doi:10.2463/mrms.mp.2019-0061
bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500,
                  1000, 1200, 1400, 1600, 1800, 2000]) # The algorithm will ignore the bvalue = 0.
std = 0.1 
n = 5 # = n-r (number of reduced b-values)

shape = [100,100,1,len(bvals)] # 4D matrix 100x100x1x15 bvals - 10 000 "voxels".

# Create a simulate signal
sig = simul_signal4d(params_mean, std, shape, bvals)

# RUN LINEAR ALGORITHM - Rsquared
best_bvals = linear_reduction(n, bvals, sig)

print(best_bvals)

In [ ]:
# Simulations Configs
params_mean = [0.00081, 0.022, 0.1] # Mean values of Gray Matter from doi:10.2463/mrms.mp.2019-0061
bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500,
                  1000, 1200, 1400, 1600, 1800, 2000]) # The algorithm will ignore the bvalue = 0.
std = 0.1 
n = 5 # = n-r (number of reduced b-values)

shape = [100,100,1,len(bvals)] # 4D matrix 100x100x1x15 bvals - 10 000 "voxels".

# Create a simulate signal
sig = simul_signal4d(params_mean, std, shape, bvals)

# RUN LINEAR ALGORITHM - Rsquared
best_bvals = linear_reduction(n, bvals, sig)

print(best_bvals)

In [ ]:
import numpy as np
from fits.nonlinfit import nonlinearfit
from autograd import grad, jacobian

import sympy
from sympy.functions import exp

In [ ]:
def gradient(bvals, d, d_star, pfraction):
    ''' Implement gradient of the function
        Parameters
    ----------
        Returns
    -------
        References
    ----------
        [1] - 
    '''
    
    col1 = -np.exp(-bvals*d)+np.exp(-bvals*d_star)
    col2 = -(1-pfraction)*bvals*np.exp(-bvals*d)
    col3 = -pfraction*bvals*np.exp(-bvals*d_star)

    Gradient = np.matrix([col1, col2, col3]).T

    return Gradient


def Hmatrix(gradient):
    ''' Implement Cook Distance for linear case
        Parameters
        ----------
        Returns
        -------
        References
        ----------
        [1] - 
    '''

    H = gradient @ np.linalg.inv(gradient.T @ gradient) @ gradient.T

    return H

def rstudentized(S, bvals, d, d_star, pfraction):
    ''' Implement Cook Distance for linear case
        Parameters
        ----------
    
        Returns
        -------
    
        References
        ----------
        [1] - 
    '''

    #Calculate ri (residual)
    ri = residual(S, bvals, d, d_star, pfraction)

    #Calculate sigma ()
    sig = np.std(ri)

    #Calculate gradiente ()
    grad = gradient(bvals, d, d_star, pfraction)

    #Calculate hii ()
    h = Hmatrix(grad)

    rst = np.zeros(len(h))

    for i in range(0,len(h)):
        rst[i]=ri[i]/(sig*np.sqrt(1-h[i,i]))
    
    return rst

def cooksdistancelinear(p, S, bvals, params):
    ''' Implement Cook Distance for linear case
        Parameters
        ----------
        Returns
        -------
        References
        ----------
        [1] - Identifying Influential Observations in Nonlinear Regression 
            - a focus on parameter estimates and the score test. Karin Stål
    '''
    
    S = S/S[0]
    
    d, d_star, pfraction = params
    
    #Calculate gradiente ()
    grad = gradient(bvals, d, d_star, pfraction)

    #Calculate h ()
    h = Hmatrix(grad)

    #Calculate t (studentized residual)
    t = rstudentized(S, bvals, d, d_star, pfraction)

    cooksdistancelinear = np.zeros(len(bvals))

    #Calculate Cooksdistance
    for i in range(len(bvals)):
        cooksdistancelinear[i] = ((t[i]**2) / p)*(h[i,i] / (1-h[i,i]))

    return cooksdistancelinear


In [1]:
# Importing Linear and Combinatory Algorithm
from algorithms.algorithms import linear_reduction, combinatory_reduction

# Import Simulation Modules
import numpy as np

# Simulation
def simul_signal4d(params_mean, std, shape, bvals):
    """ 4D Matrix are equivalent to a real image .nii
    """

    d_mean, pd_mean, f_mean = params_mean
    data = np.zeros((shape[0],shape[1],shape[2],shape[3]))
    signal = (1-f_mean)*np.exp(-bvals*d_mean)+f_mean*np.exp(-bvals*pd_mean)
    
    for i in range(shape[0]):
            for j in range(shape[1]):
                    for k in range(shape[2]):
                        noise = np.random.normal(1, std, len(signal))
                        data[i,j,k,:] = (signal*noise)
    return data

# Simulations Configs
params_mean = [0.00081, 0.022, 0.1] # Mean values of Gray Matter from doi:10.2463/mrms.mp.2019-0061
bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500,
                  1000, 1200, 1400, 1600, 1800, 2000]) # The algorithm will ignore the bvalue = 0.
std = 0.1 
n = 5 # = n-r (number of reduced b-values)

shape = [10,1,1,len(bvals)] # 4D matrix 100x100x1x15 bvals - 10 000 "voxels".

# Create a simulate signal
sig = simul_signal4d(params_mean, std, shape, bvals)

#print(sig[0,0,0])

# RUN LINEAR ALGORITHM - Rsquared
best_bvals = linear_reduction(n, bvals, sig)

print(best_bvals)

  2%|▏         | 10/600 [00:00<00:11, 50.69it/s]

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  3.86459990e-04  7.64720748e-04  1.49703867e-03
   2.70296335e-03  4.97975912e-03  8.40545587e-03  1.17692796e-02
   9.13313312e-03 -2.56400364e-03 -4.24274808e-03 -3.43505626e-03
  -3.15558520e-04  4.72490609e-03]
 [ 0.00000000e+00  7.64720748e-04  1.51323212e-03  2.96240546e-03
   5.34893648e-03  9.85533303e-03  1.66381133e-02  2.33087043e-02
   1.81244817e-02 -5.00316640e-03 -8.34183859e-03 -6.77240742e-03
  -6.37768099e-04  9.29157379e-03]
 [ 0.00000000e+00  1.49703867e-03  2.96240546e-03  5.79964350e-03
   1.04726426e-02  1.92988970e-02  3.25932032e-02  4.57085531e-02
   3.56873904e-02 -9.51454453e-03 -1.61170388e-02 -1.31589871e-02
  -1.30109350e-03  1.79597095e-02]
 [ 0.00000000e+00  2.70296335e-03  5.34893648e-03  1.04726426e-02
  

  4%|▍         | 24/600 [00:00<00:09, 58.72it/s]

[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.0743917   0.12168908  0.16145291  0.14175522  0.03139624
  -0.05355733 -0.05132294 -0.02827254 -0.00042395  0.00596083  0.01047013
   0.01351848  0.01649796]
 [ 0.          0.12168908  0.19944271  0.26591886  0.23680893  0.06104412
  -0.07608103 -0.07482058 -0.04132999 -0.00083446  0.00845756  0.01502486
   0.01946932  0.02382754]
 [ 0.          0.16145291  0.26591886  0.35896649  0.33088934  0.11382426
  -0.06185224 -0.06825694 -0.0380544  -0.00142245  0.00700666  0.01297857
   0.017035    0.021056  ]
 [ 0.          0.14175522  0.23680893  0.33088934  0.33317218  0.18381953
   0.04564015  0.01957245  0.00999924 -0.00126729 -0.00379946 -0.00555656
  -0.00671221 -0.00774823]
 [ 0.          0.03139624  0.06104412  0.11382426  0.18381953  0.25722141
   0.26851531  0.21100997  0.11719224  0.00394963 -0.02209194 -0.04

  6%|▋         | 38/600 [00:00<00:09, 59.84it/s]

...

erro
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.01030607  0.0193927   0.0516685   0.06338603  0.04115863
  -0.01567078 -0.03026337 -0.00809764 -0.00197162  0.00261764  0.00597439
   0.00835159  0.00995611]
 [ 0.          0.0193927   0.03649766  0.0973595   0.11971887  0.07854592
  -0.02775843 -0.05560924 -0.01495246 -0.00369146  0.00474604  0.01091882
   0.0152916   0.01824441]
 [ 0.          0.0516685   0.0973595   0.26174138  0.32666874  0.22820673
  -0.04414191 -0.125056   -0.03480391 -0.00937846  0.00969434  0.02366781
   0.03358671  0.04030572]
 [ 0.          0.06338603  0.11971887  0.32666874  0.41904611  0.32497317
   0.01681555 -0.09816583 -0.03012315 -0.00976506  0.00555625  0.01682454
   0.02486659  0.03035956]
 [ 0.          0.04115863  0.07854592  0.22820673  0.32497317  0.34119961
   0.21663283  0.09775552  0.01999965  0.00214378 -0.01114263 

 23%|██▎       | 140/600 [00:02<00:07, 61.45it/s]

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  9.95318276e-01  6.05823593e-02 -1.69059771e-02
  -1.64807490e-02 -1.47352001e-02 -1.13689535e-02 -6.26625260e-03
  -3.29386174e-05  1.42127807e-03  2.46040953e-03  3.17341182e-03
   3.63229065e-03  3.89512970e-03]
 [ 0.00000000e+00  6.05823593e-02  2.16025103e-01  2.23210865e-01
   2.11461693e-01  1.89061424e-01  1.45863414e-01  8.03827611e-02
   3.95843448e-04 -1.82640139e-02 -3.15971355e-02 -4.07451225e-02
  -4.66320430e-02 -5.00032964e-02]
 [ 0.00000000e+00 -1.69059771e-02  2.23210865e-01  2.37218684e-01
   2.25041772e-01  2.01738707e-01  1.56765452e-01  8.84756770e-02
   4.67889146e-03 -1.50115437e-02 -2.91681244e-02 -3.89706074e-02
  -4.53745400e-02 -4.91512539e-02]
 [ 0.00000000e+00 -1.64807490e-02  2.11461693e-01  2.25041772e-01
  

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

 23%|██▎       | 140/600 [00:19<00:07, 61.45it/s]

In [8]:
###########################################################
#                                                         #
#               NNLS Fitting scripts                      #
#                                                         #
###########################################################

import numpy as np
from scipy.optimize import nnls
from scipy.signal import find_peaks
from tqdm import tqdm


def simul_signal(params_mean, std, bvals):
    """
    """

    d_mean, pd_mean, f_mean = params_mean

    signal = (1-f_mean)*np.exp(-bvals*d_mean)+f_mean*np.exp(-bvals*pd_mean)

    noise = np.random.normal(1, std, len(signal))

    signal_with_noise = signal+(signal*noise)

    return signal_with_noise

def nnlsfit(data, bvals, region=None):
    """ Implement Non Negative Fitting for IVIM

        Parameters
    ----------
    data: array-like, 4D Matrix
    bvals: array-like
    region: array-like, [(x1,x2),(y1,y2),(z1,z2)]
        optional (default= None)

        Returns
    -------
    d_star: 3D Matrix
        The Pseudo Diffusion Coefficient.
    d: 3D Matrix
        The Diffusion Coefficient.
    pfraction: 3D Matrix
        The Perfusion Fraction.

        References
    ----------
        [1] -
    """

    if(len(data.shape) == 1):
        # Alocate memory
        s = np.zeros(len(data))
        d_star = np.zeros(1)
        d = np.zeros(1)
        f = np.zeros(1)

        # prepare for NNLS
        NUMEL_D = 400
        DIFS = 10**(np.linspace(-3, 3, NUMEL_D))/1000
        M = np.zeros([len(bvals), 400])
        for i, bval in enumerate(bvals):
            for j, dif in enumerate(DIFS):
                M[i, j] = np.exp(-bval*dif)
        
        # NNLS Fitting
        if data[0] != 0:
            s = data[:]/data[0]
            f1, f2 = 0, 0
            sk = nnls(M, s)
            sk[0][0:25] = 0
            sk[0][375:400] = 0

            pks, _ = find_peaks(np.concatenate([[min(sk[0])], sk[0],  
                                                    [min(sk[0])]]))
            pks = pks-1

            for _, pk in enumerate(pks):

                sort = sorted(sk[0][pks], reverse=True)

                if sk[0][pk] == np.amax(sk[0][pks]):
                    d = DIFS[pk]
                    f1 = sk[0][pk]

                elif len(sort) >= 2 and sk[0][pk] == sort[1]:
                    d_star = DIFS[pk]
                    f2 = np.sum(sk[0][pk-25:pk+25])
            if (f1 + f2) == 0:
                f = 0
            else:
                f = f2/(f1 + f2)

        return d_star, d, f        
    else:
        # verify the region and set the bounds of iteration
        if region is None:
            x1, x2 = 0, data.shape[0]-1
            y1, y2 = 0, data.shape[1]-1
            z1, z2 = 0, data.shape[2]-1
        else:
            x1, x2 = region[0][0], region[0][1]
            y1, y2 = region[1][0], region[1][1]
            z1, z2 = region[2][0], region[2][1]

       # Alocate memory
        s = np.zeros(data.shape[3])
        d_star = np.zeros(data.shape[0:3])
        d = np.zeros(data.shape[0:3])
        f = np.zeros(data.shape[0:3])

        # prepare for NNLS
        NUMEL_D = 400
        DIFS = 10**(np.linspace(-3, 3, NUMEL_D))/1000
        M = np.zeros([len(bvals), 400])
        for i, bval in enumerate(bvals):
            for j, dif in enumerate(DIFS):
                M[i, j] = np.exp(-bval*dif)
        
        # NNLS Fitting
        for i in range(x1, x2):
            for j in range(y1, y2):
                for k in range(z1, z2):

                    # Fit
                    if data[i, j, k, 0] != 0:
                        s = data[i, j, k, :]/data[i, j, k, 0]
                        f1, f2 = 0, 0
                        sk = nnls(M, s)
                        sk[0][0:25] = 0
                        sk[0][375:400] = 0

                        pks, _ = find_peaks(np.concatenate([[min(sk[0])], sk[0],  
                                                            [min(sk[0])]]))
                        pks = pks-1

                        for _, pk in enumerate(pks):

                            sort = sorted(sk[0][pks], reverse=True)

                            if sk[0][pk] == np.amax(sk[0][pks]):
                                d[i, j, k] = DIFS[pk]
                                f1 = sk[0][pk]

                            elif len(sort) >= 2 and sk[0][pk] == sort[1]:
                                d_star[i, j, k] = DIFS[pk]
                                f2 = np.sum(sk[0][pk-25:pk+25])
                        if (f1 + f2) == 0:
                            f[i, j, k] = 0
                        else:
                            f[i, j, k] = f2/(f1 + f2)

        return d_star, d, f

def test_fit():
    
    # define parameters of test
    params_mean = [0.001, 0.01, 0.2]
    bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500,
                      1000, 1200, 1400, 1600, 1800, 2000])
    snr = 0.1
    n = 5
    
    # Create a simulate signal
    sig = simul_signal(params_mean, snr, bvals)
    
    print(nnlsfit(sig, bvals, region=None))
    

test_fit()

 23%|██▎       | 14/60 [03:18<10:52, 14.19s/it]

(0.008124084577862966, 0.0009170774506742931, 0.33419914524396593)


In [46]:
def simul_signal3d(params_mean, std, shape, bvals):
    """
    """

    d_mean, pd_mean, f_mean = params_mean
    data = np.zeros((shape[0],shape[1],shape[2],shape[3]))
    signal = (1-f_mean)*np.exp(-bvals*d_mean)+f_mean*np.exp(-bvals*pd_mean)
    
    for i in range(shape[0]):
            for j in range(shape[1]):
                for k in range(shape[2]):
                    noise = np.random.normal(1, std, len(signal))
                    data[i,j,k,:] = signal*noise

    return data

def test_fit4D():
    
    # define parameters of test
    params_mean = [0.001, 0.01, 0.1]
    bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500,
                      1000, 1200, 1400, 1600, 1800, 2000])
    snr = 0.1
    n = 5
    shape = [32,32,1,len(bvals)]
    # Create a simulate signal
    sig = simul_signal3d(params_mean, snr, shape, bvals)
    
    results = []
    
    MAXCOUNT = np.prod(shape[0:2])
    bar = tqdm(total=MAXCOUNT, position=0)
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            for k in range(shape[2]):
                results.append(nnlsfit(sig[i,j,k], bvals, region=None))
                bar.update()
                
    bar.close()
    print(np.mean(results[0][:]))
    

test_fit4D()

 71%|███████▏  | 730/1024 [00:11<00:04, 62.68it/s]

KeyboardInterrupt: 

In [62]:
import numpy as np

def simul_signal3d(params_mean, std, shape, bvals):
    """
    """

    d_mean, pd_mean, f_mean = params_mean
    data = np.zeros((shape[0],shape[1],shape[2],shape[3]))
    signal = (1-f_mean)*np.exp(-bvals*d_mean)+f_mean*np.exp(-bvals*pd_mean)
    
    for i in range(shape[0]):
            for j in range(shape[1]):
                    for k in range(shape[2]):
                        noise = np.random.normal(1, std, len(signal))
                        data[i,j,k,:] = signal*noise

    return data
    
def data4D():
    
    # define parameters of test
    # D = 0.00081, D* = 0.022 and f = 0.2
    params_mean = [0.00081, 0.022, 0.2]
    bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500,
                      1000, 1200, 1400, 1600, 1800, 2000])
    snr = 0.25
    n = 5
    shape = [100,100,1,len(bvals)]
    # Create a simulate signal
    sig = simul_signal3d(params_mean, snr, shape, bvals)
    
    loc = '/home/lcscosta/Projetos/Ithaca/scripts/data/'
    
    np.save('data/sim_data.npy', sig)


In [63]:
data4D()

In [78]:
from tqdm import tqdm
from subsampling.subsampling import subsampling, bvals_selection_image
from fits.nnls_fit import nnlsfit

def linear_reduction(n, bvals):
    """
    Params:
        - n the number of bvals reduction
    """
    bvals_possible_reduction = bvals.tolist()
    
    data = np.load('data/sim_data.npy')
    
    print(data.shape)
    
    # Create Bar of progress
    MAXCOUNT = sum([*range(len(bvals)-n,len(bvals),1)])*np.prod(data.shape[0:2])
    bar = tqdm(total=MAXCOUNT, position=0)


    bvals_possible_reduction =  bvals_possible_reduction[1::]
    n_reduced = len(bvals_possible_reduction) - 1 

    bvals_subsampling = subsampling(n_reduced, bvals_possible_reduction)


    cookdistance_list = []
    rsqred_list = np.zeros(len(bvals_subsampling))

    # Iter in bvals_subsampling
    #   from fits/nnls_fit.py
    for j in range(len(bvals_subsampling)):

        bvals_subsampling[j].insert(0,0)

        data_subsampling = bvals_selection_image(data, bvals_subsampling[j], bvals)
        print(data_subsampling[1,1,0,:].shape)
        print(np.array(bvals_subsampling[j]).shape)
        d = np.zeros(data.shape[0:3])
        d_star = np.zeros(data.shape[0:3])
        f = np.zeros(data.shape[0:3])
        for k in range(data.shape[0]):
            for l in range(data.shape[1]):
                for m in range(data.shape[2]):
                    d_star[k,l,m], d[k,l,m], f[k,l,m] = nnlsfit(data_subsampling[k,l,m,:], bvals_subsampling[j], region=None)
                    bar.update()    
        params = [d,d_star,f]
        np.save('data/linear/'+str(bvals_subsampling[j])+'_maps.npy', params)

In [79]:
bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500,
                  1000, 1200, 1400, 1600, 1800, 2000])
n = 1
linear_reduction(1, bvals)

  0%|          | 0/140000 [00:00<?, ?it/s]

(100, 100, 1, 15)
(14,)
(14,)


  7%|▋         | 10020/140000 [01:05<14:53, 145.39it/s]

(14,)
(14,)


  8%|▊         | 10676/140000 [01:10<15:04, 143.02it/s]

KeyboardInterrupt: 

In [3]:
from math import factorial
import numpy as np

def combinatory_reduction(n, bvals):
    """
    Params:
        - n the number of bvals reduction
    """
    bvals_possible_reduction = bvals.tolist()

    data = np.load('data/sim_data.npy')

    # Create a new list of bvals
    #   from subsampling/subsampling.py
    bvals_possible_reduction = bvals_possible_reduction[1::]
    n_reduced = len(bvals_possible_reduction) - n
     
    bvals_subsampling = subsampling(n_reduced, bvals_possible_reduction)

   
    # Create Bar of progress
    MAXCOUNT = int(factorial(len(bvals_possible_reduction))/
                   (factorial(len(bvals_possible_reduction)-n)*factorial(n)))
    bar = tqdm(total=MAXCOUNT*np.prod(data.shape[0:3]), position=0)
    
    # Iter in bvals_subsampling
    #   from fits/nnls_fit.py
    for j in range(len(bvals_subsampling)):
        # save Params in .npy for every iter
        bvals_subsampling[j].insert(0,0)
        data_subsampling = bvals_selection_image(data, bvals_subsampling[j], bvals)
        
        d = np.zeros(data.shape[0:3])
        d_star = np.zeros(data.shape[0:3])
        f = np.zeros(data.shape[0:3])
        for k in range(data.shape[0]):
            for l in range(data.shape[1]):
                for m in range(data.shape[2]):
                    d_star[k,l,m], d[k,l,m], f[k,l,m] = nnlsfit(data_subsampling[k,l,m,:], bvals_subsampling[j], region=None)
                    bar.update()
        params = [d,d_star,f]
        np.save('data/combinatory/'+str(bvals_subsampling[j])+'_maps.npy', params)
    bar.close()

    
bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500,
                  1000, 1200, 1400, 1600, 1800, 2000])
n = 5
combinatory_reduction(n, bvals)

NameError: name 'subsampling' is not defined

In [4]:
maps = np.load('data/combinatory/[0, 4, 8, 16, 30, 60, 120, 250, 500, 1000]_maps.npy')

In [11]:
d, d_star, f = maps

f.shape

(100, 100, 1)

In [110]:
np.mean(d)

0.0013798389256910984

In [103]:
d =  np.load('data/linear/[0, 4, 8, 16, 30, 60, 120, 250, 500, 1000, 1200, 1400, 1600, 1800]_d.npy')

In [104]:
d.shape

(100, 100, 1)

In [105]:
np.mean(d)

0.001002782507184851

In [5]:
data = np.load('data/sim_data.npy')

In [6]:
data.shape

(100, 100, 1, 15)

In [8]:
print(data)

[[[[0.86981257 0.90118917 1.03358633 ... 0.16589158 0.25400264
    0.22173863]]

  [[0.75557716 1.31445683 0.62978573 ... 0.11681233 0.21920004
    0.1518024 ]]

  [[1.27552276 1.18127914 1.01021328 ... 0.30535295 0.21000198
    0.16986455]]

  ...

  [[0.91313439 0.98257306 1.00063609 ... 0.14571213 0.13016866
    0.1080663 ]]

  [[1.03642654 0.74389774 1.01181399 ... 0.18759839 0.19067662
    0.10626471]]

  [[0.57282433 0.53076473 1.07957309 ... 0.26765207 0.15892634
    0.2203555 ]]]


 [[[0.4196152  0.93941869 0.77416849 ... 0.21747907 0.24872783
    0.12929718]]

  [[1.03233583 0.87107979 1.05569965 ... 0.1873308  0.18248505
    0.1398178 ]]

  [[0.68503098 0.94616399 1.20849608 ... 0.13360456 0.19819929
    0.19953605]]

  ...

  [[1.13850898 1.08980332 0.85413661 ... 0.2246622  0.20867223
    0.07594029]]

  [[0.52415657 1.17596033 0.58617767 ... 0.20670968 0.11951245
    0.17789527]]

  [[1.0789375  0.97019132 0.87797714 ... 0.20747637 0.19103164
    0.13454757]]]


 [[[1.5569

In [13]:
np.mean(d_star)

0.0

In [65]:
data.shape

(100, 100, 1, 15)

In [67]:
data[0,0,0,:]

array([0.92251296, 1.03480614, 1.06874545, 0.71663942, 0.88129033,
       0.84747317, 0.76195939, 0.70197792, 0.52826597, 0.36218626,
       0.25829971, 0.27827828, 0.17410627, 0.19892635, 0.13890798])

In [25]:
import os

files_names = os.listdir('data/linear')

d, d_star, f = np.load('data/linear/'+files_names[0])

li = list(files_names[0][1:-10].split(", "))

li = [int(a) for a in li ]

In [30]:

    #li = list(files_names[0][1:-10].split(", "))

    #li = [int(a) for a in li ]
d_sub, pd_sub, f_sub = load_all('data/linear/')
d_sub.shape

(3, 100, 100, 1, 14)


(100, 100, 1, 14)

In [1]:
# Relative Root Mean Square Error (RRMSE)

In [12]:
import os

def load_all(loc):
    
    filenames = os.listdir(loc)
    
    d, d_star, f = np.load(loc+filenames[0])

    data_subsampling = np.zeros((3, d.shape[0],d.shape[1],d.shape[2],len(filenames)))
    
    for i in range(len(filenames)):
        data_subsampling[:,:,:,:,i] = np.load(loc+filenames[i])
    
    
#    print(data_subsampling.shape)
    
    return data_subsampling


def rrmse(data, data_subloc):
    """
        - Params:
            Data - 3D
            Data Subsampling - 3D x n
            
        - Return:
            - Array 3 x n Output rrmse of n images
            
        - Reference:
        [1] - DOI:10.1007/s10334-019-00764-0
    """
    
    d, pd, f = data
    
    d_sub, pd_sub, f_sub = load_all(data_subloc)
    
    rrmseD= np.zeros(d_sub.shape[-1])
    rrmsepD= np.zeros(d_sub.shape[-1])
    rrmsef= np.zeros(d_sub.shape[-1])
    
    
    for i in range(d_sub.shape[-1]):
        rrmseDsum = 0
        rrmsepDsum = 0
        rrmsefsum = 0
        for j in range(d_sub.shape[0]):
            for k in range(d_sub.shape[1]):
                for l in range(d_sub.shape[2]):
                    #print(d[j,k,l])
                    #print(d_sub[j,k,l,i])
                    #print((d[j,k,l]-d_sub[j,k,l,i])**2)
                    #print(rrmsepDsum)
                    rrmseDsum += (d[j,k,l]-d_sub[j,k,l,i])**2
                    rrmsepDsum += (pd[j,k,l]-pd_sub[j,k,l,i])**2
                    rrmsefsum += (f[j,k,l]-f_sub[j,k,l,i])**2
    
        #print(rrmsepDsum)
        #print(np.prod(d_sub[:,:,:,i].shape[0:3]))
        #print(np.mean(pd_sub[:,:,:,i]))
        #print(np.mean(pd[:,:,:]))
        rrmseD[i] = np.sqrt(rrmseDsum / np.prod(d_sub[:,:,:,i].shape[0:3]))/np.mean(d_sub[:,:,:,i])
        rrmsepD[i] = np.sqrt(rrmsepDsum / np.prod(pd_sub[:,:,:,i].shape[0:3]))/np.mean(pd_sub[:,:,:,i])
        rrmsef[i] = np.sqrt(rrmsefsum / np.prod(f_sub[:,:,:,i].shape[0:3]))/np.mean(f_sub[:,:,:,i])
        #print(rrmseDsum)
        #print(rrmsepDsum)
        #print(rrmsefsum)
        #print(np.prod(f_sub[:,:,:,i].shape[0:3]))
        #print(np.sqrt(rrmsepDsum / np.prod(pd_sub[:,:,:,i].shape[0:3])))
        #print(np.mean(pd_sub[:,:,:,i]))
        
        
        #print(np.mean(rrmseD[i]),'+/-',np.std(rrmseD[i]))
        #print(np.mean(rrmsepD[i]),'+/-',np.std(rrmsepD[i]))
        #print(np.mean(rrmsef[i]),'+/-',np.std(rrmsef[i]))
        #print(np.mean(rrmsef[i]),'+/-',np.std(rrmsef[i]))

    return [rrmseD,rrmsepD,rrmsef]

In [13]:
def select_minrrmse(maps_original, loc):
    rrmse_values = rrmse(maps_original, loc)
    min_rrmse = []
    for i in range(len(rrmse_values[0])):
        min_rrmse.append((rrmse_values[0][i]+rrmse_values[1][i]+rrmse_values[2][i])/3)
    #print(min_rrmse)
    idx = min_rrmse.index(min(min_rrmse))
    filenames = os.listdir(loc)
    file = filenames[idx][1:-10]
    
    li = list(file.split(", "))
    li = [int(a) for a in li ]
    
    return li
    
#select_minrrmse(maps,'data/linear/')

5

In [37]:
data = np.load('data/sim_data.npy')

from fits.nnls_fit import nnlsfit
d = np.zeros(data.shape[0:3])
d_star = np.zeros(data.shape[0:3])
f = np.zeros(data.shape[0:3])

bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500,
                  1000, 1200, 1400, 1600, 1800, 2000])

for k in range(data.shape[0]):
    for l in range(data.shape[1]):
        for m in range(data.shape[2]):
            d_star[k,l,m], d[k,l,m], f[k,l,m] = nnlsfit(data[k,l,m,:], bvals, region=None)
            
maps = [d, d_star, f]



In [44]:
d, d_star, f = maps 
d.shape

(100, 100, 1)

In [280]:
import matplotlib.pyplot as plt
import numpy as np
from dipy.reconst.ivim import IvimModel
from dipy.core.gradients import gradient_table, generate_bvecs
from dipy.data import get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti_data

from dipy.segment.mask import median_otsu

import nibabel as nib
import sys

bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500, 1000, 1200, 1400, 1600, 1800, 2000])

bvals_subsampling = subsampling(len(bvals)-2, bvals[1::].tolist())

print(bvals_subsampling)

#.nii load
data = abs(np.load('data/sim_data.npy'))

for i in range(len(bvals_subsampling)):
    bvals_subsampling[i].insert(0,0)
    #print(bvals_subsampling[0])
    bvecs = generate_bvecs(len(bvals_subsampling[i]))
    
    print(bvecs.shape)
    
    #print(data.shape)
    print(bvals_subsampling[i])
    print(bvals.shape)
    
    data_sub = bvals_selection_image(data, bvals_subsampling[i], bvals)
    print(data_sub.shape)

    gtab = gradient_table(bvals_subsampling[i], bvecs, b0_threshold=0)
    print('data.shape (%d, %d, %d, %d)' % data_sub.shape)

    ivimmodel = IvimModel(gtab, fit_method='trr')

    """
    To fit the model, call the `fit` method and pass the data for fitting.
    """
    print(-np.log(np.array(data_sub[0,0,0,gtab.bvals >= 200], dtype=float)))
    print(data[0,0,0,:])
    ivimfit = ivimmodel.fit(np.array(data_sub,dtype=float))

    """
    The fit method creates a IvimFit object which contains the
    parameters of the model obtained after fitting. These are accessible
    through the `model_params` attribute of the IvimFit object.
    The parameters are arranged as a 4D array, corresponding to the spatial
    dimensions of the data, and the last dimension (of length 4)
    corresponding to the model parameters according to the following
    order : $\mathbf{S_{0}, f, D^*, D}$.
    """

    ivimparams = ivimfit.model_params

    ivimparams[np.isnan(ivimparams)] = 0

    """
    As we see, we have a 20x20 slice at the height z = 33. Thus we
    have 400 voxels. We will now plot the parameters obtained from the
    fit for a voxel and also various maps for the entire slice.
    This will give us an idea about the diffusion and perfusion in
    that section. Let(i, j) denote the coordinate of the voxel. We have
    already fixed the z component as 33 and hence we will get a slice
    which is 33 units above.
    """

    estimated_params = ivimfit.model_params[:, :, :]


[[4, 8, 16, 30, 60, 120, 250, 500, 1000, 1200, 1400, 1600, 1800], [4, 8, 16, 30, 60, 120, 250, 500, 1000, 1200, 1400, 1600, 2000], [4, 8, 16, 30, 60, 120, 250, 500, 1000, 1200, 1400, 1800, 2000], [4, 8, 16, 30, 60, 120, 250, 500, 1000, 1200, 1600, 1800, 2000], [4, 8, 16, 30, 60, 120, 250, 500, 1000, 1400, 1600, 1800, 2000], [4, 8, 16, 30, 60, 120, 250, 500, 1200, 1400, 1600, 1800, 2000], [4, 8, 16, 30, 60, 120, 250, 1000, 1200, 1400, 1600, 1800, 2000], [4, 8, 16, 30, 60, 120, 500, 1000, 1200, 1400, 1600, 1800, 2000], [4, 8, 16, 30, 60, 250, 500, 1000, 1200, 1400, 1600, 1800, 2000], [4, 8, 16, 30, 120, 250, 500, 1000, 1200, 1400, 1600, 1800, 2000], [4, 8, 16, 60, 120, 250, 500, 1000, 1200, 1400, 1600, 1800, 2000], [4, 8, 30, 60, 120, 250, 500, 1000, 1200, 1400, 1600, 1800, 2000], [4, 16, 30, 60, 120, 250, 500, 1000, 1200, 1400, 1600, 1800, 2000], [8, 16, 30, 60, 120, 250, 500, 1000, 1200, 1400, 1600, 1800, 2000]]
(14, 3)
[0, 4, 8, 16, 30, 60, 120, 250, 500, 1000, 1200, 1400, 1600, 1800]

/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. Returning parameters from linear fit
  warnings.wa

KeyboardInterrupt: 

(15, 3)
data.shape (100, 100, 1, 15)
[[[[0.86981257 0.90118917 1.03358633 ... 0.16589158 0.25400264
    0.22173863]]

  [[0.75557716 1.31445683 0.62978573 ... 0.11681233 0.21920004
    0.1518024 ]]

  [[1.27552276 1.18127914 1.01021328 ... 0.30535295 0.21000198
    0.16986455]]

  ...

  [[0.91313439 0.98257306 1.00063609 ... 0.14571213 0.13016866
    0.1080663 ]]

  [[1.03642654 0.74389774 1.01181399 ... 0.18759839 0.19067662
    0.10626471]]

  [[0.57282433 0.53076473 1.07957309 ... 0.26765207 0.15892634
    0.2203555 ]]]


 [[[0.4196152  0.93941869 0.77416849 ... 0.21747907 0.24872783
    0.12929718]]

  [[1.03233583 0.87107979 1.05569965 ... 0.1873308  0.18248505
    0.1398178 ]]

  [[0.68503098 0.94616399 1.20849608 ... 0.13360456 0.19819929
    0.19953605]]

  ...

  [[1.13850898 1.08980332 0.85413661 ... 0.2246622  0.20867223
    0.07594029]]

  [[0.52415657 1.17596033 0.58617767 ... 0.20670968 0.11951245
    0.17789527]]

  [[1.0789375  0.97019132 0.87797714 ... 0.20747637 0.19

/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. Returning parameters from linear fit
  warnings.wa

In [241]:
f = ivimparams[:,:,:,1]
pd = ivimparams[:,:,:,2]
d = ivimparams[:,:,:,3]


In [130]:
np.mean(f)

0.14943306276669596

In [131]:
np.mean(pd)

0.024740973463243093

In [336]:
from tqdm import tqdm
from subsampling.subsampling import subsampling, bvals_selection_image
from fits.nnls_fit import nnlsfit
import numpy as np


def linear_fit(n, bvals):
    """
    Params:
        - n the number of bvals reduction
    """
    bvals_possible_reduction = bvals.tolist()
    
    data = np.load('data/sim_data.npy')
    
    print(data.shape)
    print(data[0,0,0,0])
    # Create Bar of progress
    MAXCOUNT = sum([*range(len(bvals)-n,len(bvals),1)])
    bar = tqdm(total=MAXCOUNT, position=0)


    bvals_possible_reduction =  bvals_possible_reduction[1::]
    n_reduced = len(bvals_possible_reduction) - 1 

    bvals_subsampling = subsampling(n_reduced, bvals_possible_reduction)
        
    # Iter in bvals_subsampling
    #   from fits/nnls_fit.py
    for j in range(len(bvals_subsampling)):

        bvals_subsampling[j].insert(0,0)
        
        data_subsampling = bvals_selection_image(data, bvals_subsampling[j], bvals)
        print(data_subsampling[0,0,0,0])
        
        bvecs = generate_bvecs(len(bvals_subsampling[j]))

        #print(-np.log(data[np.array(bvals_subsampling[j]) >= 200])

        gtab = gradient_table(np.array(bvals_subsampling[j]), bvecs, b0_threshold=0)
        
        ivimmodel = IvimModel(gtab, fit_method='trr')
       
        ivimfit = ivimmodel.fit(abs(np.array(data_subsampling, dtype=float)))

        ivimparams = ivimfit.model_params
        ivimparams[np.isnan(ivimparams)] = 0

        f = ivimparams[:,:,:,1]
        pd = ivimparams[:,:,:,2]
        d = ivimparams[:,:,:,3]
        
        params = [d,d_star,f]
        
        bar.update()    
        
        np.save('data/linear/'+ str(j) + '/' +str(bvals_subsampling[j])+'_maps.npy', params)

In [337]:
bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500,
                  1000, 1200, 1400, 1600, 1800, 2000])
n = 1
linear_fit(1, bvals)

  0%|          | 0/14 [00:00<?, ?it/s]

(100, 100, 1, 15)
0.8698125650941297
0.8698125650941297


/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. Returning parameters from linear fit
  warnings.wa

0.8698125650941297


/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. Returning parameters from linear fit
  warnings.wa

...

0.8698125650941297


/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. Returning parameters from linear fit
  warnings.wa

0.8698125650941297


/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. Returning parameters from linear fit
  warnings.wa

KeyboardInterrupt: 

In [21]:
from tqdm import tqdm
from subsampling.subsampling import subsampling, bvals_selection_image
from fits.nnls_fit import nnlsfit

from dipy.reconst.ivim import IvimModel
from dipy.core.gradients import gradient_table, generate_bvecs

def linear_reduction(n, bvals):
    """
    Params:
        - n the number of bvals reduction
    """
    bvals_possible_reduction = bvals.tolist()
    
    data = np.load('data/sim_data.npy')
    
    print(data.shape)
    
    # Create Bar of progress
    MAXCOUNT = sum([*range(len(bvals)-n,len(bvals),1)])
    bar = tqdm(total=MAXCOUNT, position=0)
    
    for i in range(n):
        bvals_possible_reduction =  bvals_possible_reduction[1::]
        n_reduced = len(bvals_possible_reduction) - 1

        bvals_subsampling = subsampling(n_reduced, bvals_possible_reduction)

        # Iter in bvals_subsampling
        #   from fits/nnls_fit.py
        for j in range(len(bvals_subsampling)):

            bvals_subsampling[j].insert(0,0)
        
            data_subsampling = bvals_selection_image(data, bvals_subsampling[j], bvals)
        
            bvecs = generate_bvecs(len(bvals_subsampling[j]))

            #print(-np.log(data[np.array(bvals_subsampling[j]) >= 200])

            gtab = gradient_table(np.array(bvals_subsampling[j]), bvecs, b0_threshold=0)
        
            ivimmodel = IvimModel(gtab, fit_method='trr')
           
            ivimfit = ivimmodel.fit(abs(np.array(data_subsampling, dtype=float)))

            ivimparams = ivimfit.model_params
            ivimparams[np.isnan(ivimparams)] = 0

            f = ivimparams[:,:,:,1]
            pd = ivimparams[:,:,:,2]
            d = ivimparams[:,:,:,3]
        
            params = [d,pd,f]
        
            bar.update()    
        
            np.save('data/linear/'+ str(i) + '/' +str(bvals_subsampling[j])+'_maps.npy', params)

        bvals_possible_reduction = select_minrrmse(maps,'data/linear/'+str(i)+'/')
        
    return bvals_possible_reduction

In [66]:
bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500, 1000, 1200, 1400, 1600, 1800, 2000])
linear_reduction(5, bvals)

  0%|          | 0/60 [00:00<?, ?it/s]

(100, 100, 1, 15)


/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. Returning parameters from linear fit
  warnings.wa

 13%|█▎        | 8/60 [06:49<42:33, 49.10s/it]/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. Retu

 27%|██▋       | 16/60 [13:38<37:38, 51.33s/it]/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. Ret

...

 53%|█████▎    | 32/60 [4:09:58<2:58:30, 382.50s/it]/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting

 67%|██████▋   | 40/60 [4:16:41<22:59, 68.98s/it]/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. R

 80%|████████  | 48/60 [4:23:59<12:02, 60.22s/it]/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. R

 93%|█████████▎| 56/60 [4:31:03<03:15, 48.81s/it]/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. R

[0, 4, 8, 16, 120, 500, 1000, 1200, 1800, 2000]

In [65]:
import matplotlib.pyplot as plt
import numpy as np
from dipy.reconst.ivim import IvimModel
from dipy.core.gradients import gradient_table, generate_bvecs
from dipy.data import get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti_data

from dipy.segment.mask import median_otsu

import nibabel as nib
import sys

bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500, 1000, 1200, 1400, 1600, 1800, 2000])

bvecs = generate_bvecs(len(bvals))

gtab = gradient_table(bvals, bvecs, b0_threshold=0)


ivimmodel = IvimModel(gtab, fit_method='trr')

"""
To fit the model, call the `fit` method and pass the data for fitting.
"""
data = np.load('data/sim_data.npy')

ivimfit = ivimmodel.fit(abs(np.array(data, dtype=float)))

"""
The fit method creates a IvimFit object which contains the
parameters of the model obtained after fitting. These are accessible
through the `model_params` attribute of the IvimFit object.
The parameters are arranged as a 4D array, corresponding to the spatial
dimensions of the data, and the last dimension (of length 4)
corresponding to the model parameters according to the following
order : $\mathbf{S_{0}, f, D^*, D}$.
"""

ivimparams = ivimfit.model_params

ivimparams[np.isnan(ivimparams)] = 0

"""
As we see, we have a 20x20 slice at the height z = 33. Thus we
have 400 voxels. We will now plot the parameters obtained from the
fit for a voxel and also various maps for the entire slice.
This will give us an idea about the diffusion and perfusion in
that section. Let(i, j) denote the coordinate of the voxel. We have
already fixed the z component as 33 and hence we will get a slice
which is 33 units above.
"""

estimated_params = ivimfit.model_params[:, :, :]

f = ivimparams[:,:,:,1]
pd = ivimparams[:,:,:,2]
d = ivimparams[:,:,:,3]
    
maps = [d,pd,f]

np.save('data/maps.npy', maps)

/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. Returning parameters from linear fit
  warnings.wa

In [68]:
best_bvals = [0, 4, 8, 16, 120, 500, 1000, 1200, 1800, 2000]#[0, 4, 8, 16, 60, 1000, 1200, 1400, 1800, 2000]#[0, 4, 8, 16, 30, 60, 120, 1400, 1600, 2000] 
#[0, 4, 8, 16, 120, 500, 1000, 1200, 1800, 2000]

data_subsampling = bvals_selection_image(data, best_bvals, bvals)

import matplotlib.pyplot as plt
import numpy as np
from dipy.reconst.ivim import IvimModel
from dipy.core.gradients import gradient_table, generate_bvecs
from dipy.data import get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti_data

from dipy.segment.mask import median_otsu

import nibabel as nib
import sys


bvecs = generate_bvecs(len(best_bvals))

gtab = gradient_table(best_bvals, bvecs, b0_threshold=0)


ivimmodel = IvimModel(gtab, fit_method='trr')

"""
To fit the model, call the `fit` method and pass the data for fitting.
"""

ivimfit = ivimmodel.fit(abs(np.array(data_subsampling, dtype=float)))

"""
The fit method creates a IvimFit object which contains the
parameters of the model obtained after fitting. These are accessible
through the `model_params` attribute of the IvimFit object.
The parameters are arranged as a 4D array, corresponding to the spatial
dimensions of the data, and the last dimension (of length 4)
corresponding to the model parameters according to the following
order : $\mathbf{S_{0}, f, D^*, D}$.
"""

ivimparams = ivimfit.model_params

ivimparams[np.isnan(ivimparams)] = 0

"""
As we see, we have a 20x20 slice at the height z = 33. Thus we
have 400 voxels. We will now plot the parameters obtained from the
fit for a voxel and also various maps for the entire slice.
This will give us an idea about the diffusion and perfusion in
that section. Let(i, j) denote the coordinate of the voxel. We have
already fixed the z component as 33 and hence we will get a slice
which is 33 units above.
"""

estimated_params = ivimfit.model_params[:, :, :]

f = ivimparams[:,:,:,1]
pd = ivimparams[:,:,:,2]
d = ivimparams[:,:,:,3]
    
maps = [d,pd,f]

/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. Returning parameters from linear fit
  warnings.wa

In [67]:
print(np.mean(d),'+-',np.std(d))
print(np.mean(pd),'+-',np.std(pd))
print(np.mean(f),'+-',np.std(f))

0.0008015119477117675 +- 0.00022304269790719868
0.024298262440616777 +- 0.11515128128003584
0.1513084712946047 +- 0.3032319700939237


In [69]:
print(np.mean(d),'+-',np.std(d))
print(np.mean(pd),'+-',np.std(pd))
print(np.mean(f),'+-',np.std(f))

0.0008013628873827669 +- 0.00022975626454407397
0.03000706525472163 +- 0.1357512561972369
0.15665042390901845 +- 0.3070772260547958


In [59]:
fpos = [x for x in f.reshape(1,10000) if x>= 0]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [61]:
f


array([[[ 0.41415133],
        [-0.88155517],
        [ 0.26211406],
        ...,
        [ 0.27911002],
        [-0.15459669],
        [ 0.11458214]],

       [[ 0.72951308],
        [-0.49567264],
        [ 0.27990884],
        ...,
        [-0.0587067 ],
        [ 0.61392926],
        [ 0.58139298]],

       [[ 0.38528636],
        [-0.52319172],
        [-0.0570712 ],
        ...,
        [ 0.34912267],
        [ 0.2       ],
        [-0.0916603 ]],

       ...,

       [[-0.42061544],
        [-1.0484102 ],
        [-2.97130886],
        ...,
        [ 0.20007924],
        [-0.0486839 ],
        [ 0.2       ]],

       [[ 0.26681279],
        [ 0.48570639],
        [ 0.66179113],
        ...,
        [-0.00439516],
        [-0.77366601],
        [-0.2900912 ]],

       [[-0.13026126],
        [-2.37110472],
        [-0.25885639],
        ...,
        [-1.60716752],
        [-0.20627299],
        [ 0.18736245]]])

In [60]:
np.mean(f[f>0])

0.3467702399917961

In [393]:
import matplotlib.pyplot as plt
import numpy as np
from dipy.reconst.ivim import IvimModel
from dipy.core.gradients import gradient_table, generate_bvecs
from dipy.data import get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti_data

from dipy.segment.mask import median_otsu

import nibabel as nib
import sys

bvals = np.array([0, 4, 8, 16, 30, 60, 120, 250, 500, 1000, 1200, 1400, 1600, 1800, 2000])

#print(bvals_subsampling)

#.nii load
data = abs(np.load('data/sim_data.npy'))
    
bvecs = generate_bvecs(len(bvals))

print(bvecs.shape)

#print(data.shape)
#print(bvals_subsampling[i])
#print(bvals.shape)

#data_sub = bvals_selection_image(data, bvals_subsampling[i], bvals)
#print(data_sub.shape)

gtab = gradient_table(bvals, bvecs, b0_threshold=0)
print('data.shape (%d, %d, %d, %d)' % data.shape)

ivimmodel = IvimModel(gtab, fit_method='trr')

"""
To fit the model, call the `fit` method and pass the data for fitting.
"""
print(data)
ivimfit = ivimmodel.fit(data)

"""
The fit method creates a IvimFit object which contains the
parameters of the model obtained after fitting. These are accessible
through the `model_params` attribute of the IvimFit object.
The parameters are arranged as a 4D array, corresponding to the spatial
dimensions of the data, and the last dimension (of length 4)
corresponding to the model parameters according to the following
order : $\mathbf{S_{0}, f, D^*, D}$.
"""

ivimparams = ivimfit.model_params

ivimparams[np.isnan(ivimparams)] = 0

"""
As we see, we have a 20x20 slice at the height z = 33. Thus we
have 400 voxels. We will now plot the parameters obtained from the
fit for a voxel and also various maps for the entire slice.
This will give us an idea about the diffusion and perfusion in
that section. Let(i, j) denote the coordinate of the voxel. We have
already fixed the z component as 33 and hence we will get a slice
which is 33 units above.
"""

estimated_params = ivimfit.model_params[:, :, :]

f = ivimparams[:,:,:,1]
pd = ivimparams[:,:,:,2]
d = ivimparams[:,:,:,3]
    
maps = [d,pd,f]

(15, 3)
data.shape (100, 100, 1, 15)
[[[[0.86981257 0.90118917 1.03358633 ... 0.16589158 0.25400264
    0.22173863]]

  [[0.75557716 1.31445683 0.62978573 ... 0.11681233 0.21920004
    0.1518024 ]]

  [[1.27552276 1.18127914 1.01021328 ... 0.30535295 0.21000198
    0.16986455]]

  ...

  [[0.91313439 0.98257306 1.00063609 ... 0.14571213 0.13016866
    0.1080663 ]]

  [[1.03642654 0.74389774 1.01181399 ... 0.18759839 0.19067662
    0.10626471]]

  [[0.57282433 0.53076473 1.07957309 ... 0.26765207 0.15892634
    0.2203555 ]]]


 [[[0.4196152  0.93941869 0.77416849 ... 0.21747907 0.24872783
    0.12929718]]

  [[1.03233583 0.87107979 1.05569965 ... 0.1873308  0.18248505
    0.1398178 ]]

  [[0.68503098 0.94616399 1.20849608 ... 0.13360456 0.19819929
    0.19953605]]

  ...

  [[1.13850898 1.08980332 0.85413661 ... 0.2246622  0.20867223
    0.07594029]]

  [[0.52415657 1.17596033 0.58617767 ... 0.20670968 0.11951245
    0.17789527]]

  [[1.0789375  0.97019132 0.87797714 ... 0.20747637 0.19

/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:151: UserWarning: Bounds for this fit have been set from experiments and literature survey. To change the bounds, please input your bounds in model definition...
  warnings.warn(bounds_warning, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:434: UserWarning: x0 obtained from linear fitting is not feasibile as initial guess for leastsq while estimating f and D_star. Using parameters from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:507: UserWarning: x0 is unfeasible for leastsq fitting. Returning x0 values from the linear fit.
  warnings.warn(warningMsg, UserWarning)
/home/lcscosta/Documentos/anaconda3/lib/python3.8/site-packages/dipy/reconst/ivim.py:344: UserWarning: Bounds are violated for leastsq fitting. Returning parameters from linear fit
  warnings.wa

In [394]:
print(np.mean(d),'+-',np.std(d))
print(np.mean(pd),'+-',np.std(pd))
print(np.mean(f),'+-',np.std(f))

0.0008061425393641375 +- 0.00021997281427279554
0.024740973463243093 +- 0.1173480380522423
0.14943306276669596 +- 0.30285531961846685
